In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import json
import pickle
from datetime import date, timedelta

In [ ]:
_kamers = open("kamers.json", "r")
kamers = json.loads(_kamers.read())
_kamers.close()

_devices = open("devices.p", "rb")
devices = pickle.load(_devices)
_devices.close()

In [ ]:
one = devices['Huis']['Electriciteit_opwek_dak']
# The monitoring device I'm using _always_ reads either 0.1 or 0.2 Wh / 15 minutes
# , so only retain values above that amount
one = one[one.apply(lambda x: x['value'] > 0.2, axis=1)]

two = devices['Huis']['Electriciteit_opwek_garage']

both = pd.merge(one, two, left_index=True, right_index=True).sum(axis=1).to_frame()
both.rename(columns={both.columns[0]: 'value' }, inplace = True)

# Todo: derive from the dataset
years = ['2018', '2019', '2020']

sns.set(rc={'figure.figsize':(18, 6)})

In [ ]:
def chart_timespan(when):
    when = when.strftime('%Y-%m-%d')
    
    df = one[when].index.to_frame().drop(['_from'], axis=1)
    df['Huis'] = one[when]['value']
    df['Garage'] = two[when]['value']
    df['Beide'] = both[when]['value']

    q = df.plot.area(stacked = False)
    q.set_xlabel('Tijd op ' + when)
    q.set_ylabel('Wh / 15 minuten')

    df = df.resample('H').sum()
    h = df.plot.area(stacked = False)
    h.set_xlabel('Tijd op ' + when)
    h.set_ylabel('Wh')

    return (q, h)

In [ ]:
today = date.today()
for i in range(0, 7):
    d = today - timedelta(days=(i + 1))
    chart_timespan(d)

In [ ]:
def compare_years(source, years, x_axis = lambda d: d.index.dayofyear):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, 'value']
        x = x.map(lambda x: x / 1000, na_action=None)
        ax.plot(x_axis(x), x, marker='x', linestyle='-', linewidth=0.5, label='kWh')
    ax.set_ylabel('kWh')
    ax.legend()

def compare_years_cumsum(source, years, xlabel):
    fig, ax = plt.subplots()
    source = source.fillna(0)
    for year in years:
        x = source.loc[year]
        x = x.cumsum(axis=0)
        x = x.map(lambda x: x / 1000, na_action=None)
        lbl = year + ' end: ' + str(round(x.iloc[-1], 2)) + ' kWh'
        ax.plot(x.index.dayofyear, x, marker='.', linestyle='-', linewidth=0.5, label=lbl)
    ax.set_ylabel('kWh')
    ax.set_xlabel(xlabel)
    ax.legend()

In [ ]:
compare_years_cumsum(one['value'], years, 'Huis')
# The first 2 years there was no data anyways
compare_years_cumsum(two['value'], years[2:], 'Garage')
compare_years_cumsum(both['value'], years, 'Beide')

In [ ]:
def scatter_plot(df):
    df['day'] = df.index.dayofyear
    df['hour'] = df.index.hour
    df = df[df.apply(lambda x: x['value'] > 10, axis=1)]
    df.plot.scatter(x='day', y='hour', c='value', cmap='viridis')

In [ ]:
scatter_plot(one)

In [ ]:
scatter_plot(two)

In [ ]:
scatter_plot(both)